In [0]:
%run ./00_Setup_Config

In [0]:
# Read data using AutoLoader
df_raw = (spark.readStream
          .format("cloudFiles")
          .option("cloudFiles.format", "csv")
          .option("header", "true")
          .option("sep", ",")
          .option("inferSchema", "true")
          .option("cloudFiles.inferColumnTypes", "true")
          .option("cloudFiles.schemaLocation", checkpoint_path)
          .load(raw_data_path))

def clean_column_name(col_name):
    return col_name.replace(' ', '_').replace('(', '__').replace(')', '').replace('%', 'pct')

for col_name in df_raw.columns:
    df_raw = df_raw.withColumnRenamed(col_name, clean_column_name(col_name))

(df_raw.writeStream
    .option("checkpointLocation", checkpoint_path)
    .trigger(once=True)
    .toTable(bronze_table_name))

In [0]:
%sql
SELECT * FROM dblearning.us_accidents.bronze_accidents LIMIT 10;